In [ ]:
# Packages used in the script
import numpy as np
import pandas as pd
from pandas.io.json import json_normalize
import gzip
import spacy
import re
from contractions import contractions_dict

In [ ]:
# source: https://jmcauley.ucsd.edu/data/amazon/qa/
# Reading the data
def parse(path):
    g = gzip.open(path, 'rb')
    for l in g:
        yield eval(l)

# Convert to Pandas DataFrame        
def getDF(path):
    i = 0
    df = {}
    for d in parse(path):
        df[i] = d
        i += 1
    return pd.DataFrame.from_dict(df, orient='index')

df_sports = getDF('qa_Sports_and_Outdoors.json.gz')

In [ ]:
pd.set_option('display.max_columns', None)  
pd.set_option('display.expand_frame_repr', False)
pd.set_option('max_colwidth', -1)

In [ ]:
# Only extract the open-ended questions from the dataset
df_sports_open = df_sports[df_electronics['questionType'] == 'open-ended']

In [ ]:
# Remove the columns that are not relevant 
df_sports_amazon = df_sports_open.drop(['unixTime', 'answerType', 'answerTime'], axis=1)

In [ ]:
# Remove contraction using the contraction library 
contractions_dict.pop('em') #removed as this causes ambiguities
for contraction in contractions_dict:
    df_sports_amazon["answer"] = df_sports_amazon["answer"].apply(lambda x:str(x).replace(contraction,contractions_dict[contraction]))

In [ ]:
df_sports_amazon['answer'] = df_sports_amazon['answer'].str.lower()

In [ ]:
# Extract sentences from the answers
nlp = spacy.load("en_core_web_sm")

df_sports_amazon['sentences'] = df_sports_amazon['answer'].apply(lambda x: [str(sent).strip() for sent in nlp(str(x)).sents])

In [ ]:
 # Create lists to fill with values
l_col1 = []
l_col2 = []

# iterate over each row and fill our lists
for ix, row in df_sports_amazon.iterrows():
    for value in row['sentences']:
        l_col1.append(value)
        l_col2.append(row['asin'])
        #l_col3.append(row['answer'])

# Create new dataframe from the two lists
df_sport = pd.DataFrame({'sentences': l_col1 ,
                         'asin': l_col2})
df_sport = df_sport.rename(columns={"asin":"product_type"});

In [ ]:
# Export DataFrame to CSV file
df_sport.to_csv(r'/df_sport.csv', index = False)

References dataset:

Wan, M., & McAuley, J. (2016). Modeling ambiguity, subjectivity, and diverging viewpoints in opinion question answering systems. In 2016 IEEE 16th international conference on data mining (ICDM) (pp. 489-498). IEEE.

McAuley, J., & Yang, A. (2016). Addressing complex and subjective product-related queries with customer reviews. In Proceedings of the 25th International Conference on World Wide Web (pp. 625-635).